**1. Begin (or restart) part "3(a)" of the TUT Demo and interact with a ChatBot to make sure you understand how each part the Monte Hall problem code above works.**

In [1]:
import numpy as np
all_door_options = (1,2,3)  # tuple
my_door_choice = 1  # 1,2,3
i_won = 0
reps = 100000
for i in range(reps):
    secret_winning_door = np.random.choice(all_door_options)
    all_door_options_list = list(all_door_options)

    all_door_options_list.remove(secret_winning_door)
    try:

        all_door_options_list.remove(my_door_choice)
    except:
        pass
    # show a "goat" losing door and remove it
    goat_door_reveal = np.random.choice(all_door_options_list)
    all_door_options_list.remove(goat_door_reveal)

    if secret_winning_door != my_door_choice:
        all_door_options_list.append(secret_winning_door)

    my_door_choice = all_door_options_list[0]

    if my_door_choice == secret_winning_door:
        i_won += 1

i_won/reps

0.66649

**2. Extend your ChatBot sessions to now address part "3(b)" of the TUT Demo and interact with your ChatBot to see if it can suggest a simpler, more streamlined way to code up this for loop simulation so the process is more clear and easier to understand; then, describe any preferences you have in terms of readibility or explainability between the original code and the code improvements suggested by the ChatBot.**

In [3]:
import numpy as np

all_door_options = [1, 2, 3]
i_won = 0
reps = 100000

for _ in range(reps):
    secret_winning_door = np.random.choice(all_door_options)
    my_door_choice = 1  # Always initially pick door 1
    remaining_doors = [door for door in all_door_options if door != my_door_choice and door != secret_winning_door]
    goat_door_reveal = np.random.choice(remaining_doors)
    
    my_door_choice = [door for door in all_door_options if door != my_door_choice and door != goat_door_reveal][0]
    
    if my_door_choice == secret_winning_door:
        i_won += 1

win_rate = i_won / reps
win_rate

0.66749

Between this and the original code, honestly these two codes are not that different The only difference between the codes is the part where the contestant switches doors. I would prefer the original code for this part, as the steps are more precise and elementary in a way, which makes it more clear on what exactly the code is doing.

**3. Submit your preferred version of the Monty Hall problem that is verified to be running and working with a final printed output of the code; then, add code comments explaining the purpose of each line of the code.**

In [4]:
import numpy as np
all_door_options = (1,2,3)  # tuple
my_door_choice = 1  # 1,2,3
i_won = 0
reps = 100000
for i in range(reps):
    secret_winning_door = np.random.choice(all_door_options)
    all_door_options_list = list(all_door_options)

    all_door_options_list.remove(secret_winning_door)
    try:

        all_door_options_list.remove(my_door_choice)
    except:
        pass
    # show a "goat" losing door and remove it
    goat_door_reveal = np.random.choice(all_door_options_list)
    all_door_options_list.remove(goat_door_reveal)

    if secret_winning_door != my_door_choice:
        all_door_options_list.append(secret_winning_door)

    my_door_choice = all_door_options_list[0]

    if my_door_choice == secret_winning_door:
        i_won += 1

i_won/reps

0.66569

The for loop runs 100000 times, accordring to the code range(reps). It chooses a random door from 1, 2, 3 100000 times. Then, the all door options is turned into a list, then the winning door is removed from the list. Then it tries to remove my door choice, except for when the choice door is the winnning door. The remaining doors in the list is then randomly chosen and revealed as the losing door. Then my door choice becomes the other remaining door. If the door choice is the winning door, i_won is added once. Finally, the number of wins devided by reps is done, revealing the probability.

**4. Watch the embedded video tutorial on Markov chains in the next Jupyter cell below to understand their application and relevance for ChatBots; then, after watching the video, start a new ChatBot session by prompting that you have code that creates a "Markovian ChatBot"; show it the first version of the "Markovian ChatBot code" below; and interact with the ChatBot session to make sure you understand how the original first version of the "Markovian ChatBot code" works.**

In [7]:
// Command charchain generates text character-by-character
// by building a Markov chain on a text document.
package main

import (
	"fmt"
	"io/ioutil"
	"log"
	"math/rand"
	"os"
	"strconv"
	"strings"

	"github.com/unixpickle/markovchain"
)

func main() {
	if len(os.Args) != 3 {
		fmt.Fprintf(os.Stderr, "Usage: %s text_doc.txt history_size\n", os.Args[0])
		os.Exit(1)
	}

	historySize, err := strconv.Atoi(os.Args[2])
	if err != nil {
		fmt.Fprintln(os.Stderr, "Invalid history size:", os.Args[2])
		os.Exit(1)
	}

	body, err := ioutil.ReadFile(os.Args[1])
	if err != nil {
		fmt.Fprintln(os.Stderr, "Failed to read text:", err)
		os.Exit(1)
	}

	fields := strings.Split(string(body), "")
	fieldChan := make(chan string, 10)
	go func() {
		for _, field := range fields {
			fieldChan <- field
		}
		close(fieldChan)
	}()

	log.Println("Building chain...")
	chain := markovchain.NewChainText(fieldChan, historySize)
	log.Println("Generating text...")
	state := randomStart(chain)
	for i := 0; i < 1000; i++ {
		ts := state.(markovchain.TextState)
		fmt.Print(ts[len(ts)-1])
		state = randomTransition(chain, state)
	}
	fmt.Println("")
}

func randomStart(ch *markovchain.Chain) markovchain.State {
	var allStates []markovchain.State
	ch.Iterate(func(s *markovchain.StateTransitions) bool {
		allStates = append(allStates, s.State)
		return true
	})
	state := allStates[rand.Intn(len(allStates))]

	// Run through the markov chain to land at a more
	// likely state.
	for i := 0; i < 10; i++ {
		newState := randomTransition(ch, state)
		if newState == nil {
			break
		}
		state = newState
	}

	return state
}

func randomTransition(ch *markovchain.Chain, state markovchain.State) markovchain.State {
	entry := ch.Lookup(state)
	if entry == nil || len(entry.Targets) == 0 {
		return nil
	}

	prob := rand.Float64()
	var curProb float64
	for i, x := range entry.Probabilities {
		curProb += x
		if curProb > prob {
			return entry.Targets[i]
		}
	}

	return entry.Targets[len(entry.Targets)-1]
}

SyntaxError: invalid syntax (4047033595.py, line 1)

**5. Recreate (or resume) the previous ChatBot session from question "4" above, and now prompt the ChatBot session that you have a couple extensions of the code to show it, and then show it each of the extentions of the "Markovian ChatBot code" below in turn.**

1. Without just supplying your ChatBot session with the answers, see if the ChatBot can figure out what the extensions in the code do; namely, making character specific Markov chains, and using bigrams (rather than just the previous word alone) dependency... prompt your ChatBot session with some hints if it's not seeming to "get it".

2. Interact with your ChatBot session to have it explain details of the code wherever you need help understanding what the code is doing and how it works

3. Start yet another new ChatBot session and first show the ChatBot the original "Markovian ChatBot code" below, and then tell ChatBot that you have an extension but this time just directly provide it the more complicated final extension without ever providing the intermediate extension code to the ChatBot session and see if it's still able to understand everything extension does; namely, making character specific Markov chains, and using bigrams (rather than just the previous word alone) dependency... prompt the ChatBot with some hints if it's not seeming to understand what you're getting at...

**6. Report on your experience interacting with ChatBots to understand the Monte Hall problem and "Markovian ChatBot" code.**

1. Discuss how quickly the ChatBot was able to be helpful for each of the above questions, and if so, how?


The ChatBot quickly provided help with both the Monte Hall problem and the "Markovian ChatBot" code. It explained the probabilities and strategy for Monte Hall clearly, and offered useful suggestions for the Markov model. The responses were efficient, making the interaction smooth and productive.

2. Discuss whether or not interacting with ChatBot to try to figure things out was frustrating or unhelpful, and if so, how?

Interacting with the ChatBot wasn’t particularly frustrating, as it provided clear and relevant information. However, there were moments when I had to rephrase or clarify my questions to get the most accurate response, which could slow things down slightly. Overall, though, the assistance was helpful, and the need for adjustments didn’t outweigh the value of ChatBots.

3.  Based on your experiences to date (e.g., including using ChatBots to troubleshoot coding errors in the previous homework), provide an overall assessment evaluating the usefulness of ChatBots as tools to help you understand code

Overall, ChatBots are useful for understanding code with clear explanations. While they sometimes need specific prompts, their instant feedback is valuable, especially with more well-defined problems. It helped me a lot to understand the Monte Hall problem code and the 'Markovian ChatBot' code. ChatBots would sometimes struggle with more complex problems that require more advanced knowledge though.

**7. Reflect on your experience interacting with ChatBot and describe how your perception of AI-driven assistance tools in the context of learning coding, statistics, and data science has been evolving (or not) since joining the course.**

My views on ChatBot since starting this course has not changed that much. I have always understand how useful ChatBot can be, but I have only just started implementing more of this for the classes. There was a trend in the education community to try to avoid ChatBots as much possible, but this course encourages the use of this which is new.

**8. ChatBots consume text data available on the web or platforms, and thus represents a new way to "search consensensus" that condenses and summarizes mainstream human thought.**

1. Start a new ChatBot session and discuss the relevance of learning and adaptability, communication, coding, and statistics and data analysis as skills in the modern world, especially with respect to career opportunities (particularly in the context of the data science industry).

2. See if ChatBot thinks you could be a statistician or data scientist without coding or doing data analysis, and then transition your ChatBot conversation into a career exploration discussion, using the ChatBot to identify the skills that might be the most valuable for a career that you're interested.

3. Ask for a summary of this ChatBot session and paste it into your homework notebook (including link(s) to chat log histories if you're using ChatBot).

In this conversation, you initially asked about the relevance of specific skills such as learning and adaptability, communication, coding, and statistics in the modern world, particularly in the data science industry. I explained how these skills are interconnected and crucial in data science careers, emphasizing that learning adaptability, communication, coding, and data analysis all contribute to career success in this field.

Next, you inquired if it was possible to be a statistician or data scientist without coding or data analysis. I explained that while certain roles like advisory or managerial positions might allow you to avoid hands-on coding and analysis, having a foundational understanding of these skills is typically expected for most data-related roles.

Finally, you expressed interest in exploring careers and specifically wanted to know which skills would be most valuable for a software engineer. I provided a detailed breakdown of the essential technical and soft skills needed, including programming languages, algorithms, version control, frameworks, databases, cloud computing, problem-solving, and Agile methodologies, along with the importance of communication, collaboration, and open-source contributions.

4. Paraphrase the assessments and conclusions of your conversation in the form of a reflection on your current thoughts regarding your potential future career(s) and how you can go about building the skills you need to pursue it.

As I reflect on the recent career exploration, especially in software engineering and data science, I think one needs to possess a fusion of technical skills with soft skills. Basically, learning and adaptability provide a foundation for every modern tech career—be it knowledge about new programming languages, understanding emergent technologies, or methodology. I know I need to be flexible and willing to learn.

5. Give your thoughts regarding the helpfulness or limitations of your conversation with a ChatBot, and describe the next steps you would take to pursue this conversation further if you felt the information the ChatBot provides was somewhat high level and general, and perhaps lacked the depth and detailed knowledge of a dedicated subject matter expert who had really take the time to understand the ins and outs of the industry and career path in question.

While chatbots provide fast general thoughts and systematic advice, they lack depth, personalization, and industry-specific know-how. For more advanced knowledge, I need to find myself an industry insider, professional community, specialized course, and such. ChatBots are not all knowing for now.

**9. Have you reviewed the course wiki-textbook and interacted with a ChatBot (or, if that wasn't sufficient, real people in the course piazza discussion board or TA office hours) to help you understand all the material in the tutorial and lecture that you didn't quite follow when you first saw it?**

Yes.

Question 8: https://chatgpt.com/share/66f20613-ba9c-8001-bf7b-76b10acb06d6

Questions 1 - 3: https://chatgpt.com/share/66f20bc6-5e60-8001-bdae-8d38a8685bf3

Questions 4 - 5: https://chatgpt.com/share/66f20e3c-d970-8001-85f4-40d5457cc55b